In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
from shapely.geometry import Point

In [78]:
# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [79]:
# 데이터 불러오기
dong_gdf = gpd.read_file('BND_ADM_DONG_PG_dong/seoul_dong_2018.shp', encoding="UTF-8")
gu_gdf = gpd.read_file('LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11_202405.shp', encoding="CP949")
bus_stops = pd.read_csv('busstop_counts/busstop_counts_20242.csv')

In [80]:
# 버스 정류소 데이터를 GeoDataFrame으로 변환
bus_stops['geometry'] = bus_stops.apply(lambda x: Point(x['X좌표'], x['Y좌표']), axis=1)
bus_stops_gdf = gpd.GeoDataFrame(bus_stops, geometry='geometry', crs="EPSG:4326")

In [81]:
# 좌표계 통일 (EPSG:5181 - Korea Central Belt 2010)
dong_gdf = dong_gdf.to_crs("EPSG:5181")
gu_gdf = gu_gdf.to_crs("EPSG:5181")
bus_stops_gdf = bus_stops_gdf.to_crs("EPSG:5181")

In [82]:
# Spatial Join: 버스 정류소와 동 경계 결합
joined_dong = gpd.sjoin(bus_stops_gdf, dong_gdf, how="inner", predicate="within")

# 동별 버스 정류소 개수 계산
dong_bus_counts = joined_dong.groupby('adm_nm').size().reset_index(name='Bus_Stop_Count')

# 병합: 동 경계 데이터에 버스 정류소 개수 추가
dong_gdf = dong_gdf.merge(dong_bus_counts, how="left", on="adm_nm")
dong_gdf['Bus_Stop_Count'] = dong_gdf['Bus_Stop_Count'].fillna(0)  # NaN 값을 0으로 대체

# 동 데이터 확인
print("총 동 개수:", len(dong_gdf))
print(dong_gdf[['adm_nm', 'Bus_Stop_Count']].head())


총 동 개수: 424
          adm_nm  Bus_Stop_Count
0  서울특별시 종로구 사직동              41
1  서울특별시 종로구 삼청동              16
2  서울특별시 종로구 부암동              35
3  서울특별시 종로구 평창동              60
4  서울특별시 종로구 무악동               4


In [83]:
# csv파일로 저장
column_names = ['adm_nm', 'Bus_Stop_Count']
dong_gdf.to_csv('busstop_counts_20242.csv', index=False, columns=column_names)